In [ ]:
# 1) If a decision tree is underfitting, it it not worth it to scale our input because it does not matter if the data is scaled or centered
#    in a decision tree. 

# 2) If a decision tree is overfitting, it is a good idea to decrease max_depth because it will regularize the model. 

# 3) D

# 4) D

# 5) Gini Index = 1 - P((Yes))^2 - P((No))^2

#    Tree1Left = 1 - (2/5)^2 - (3/5)^2 = 0.48
#    Gain = -(2/5)*log2(2/5) - (3/5)*log2(3/5) = 0.97

#    Tree1Mid = 1 - (4/4)^2 - (0)^2 = 0
#    Gain = -(4/4)*log2(4/4) - (0)*log2(0) = 0

#    Tree1Right = 1 - (3/5)^2 - (2/5)^2 = 0.48
#    Gain = -(3/5)*log2(3/5) - (2/5)*log2(2/5) = 0.97
#    Tree1Total = 1.94

#    Tree2Left = 1 - (2/4)^2 - (2/4)^2 = 0.5
#    Gain = -(2/4)*log2(2/4) - (2/4)*log2(2/4) = 1

#    Tree2Mid = 1 - (4/6)^2 - (2/6)^2 = 0.44
#    Gain = -(4/6)*log2(4/6) - (2/6)*log2(2/6) = 0.92

#    Tree2Right = 1 - (3/4)^2 - (1/4)^2 = 0.375
#    Gain = -(3/4)*log2(3/4) - (1/4)*log2(1/4) = 0.812
#    Tree2Total = 2.73

#    Tree3Left = 1 - (6/8)^2 - (2/8)^2 = 0.375
#    Gain = -(6/8)*log2(6/8) - (2/8)*log2(2/8) = 0.812

#    Tree3Right = 1 - (3/6)^2 - (3/6)^2 = 0.5
#    Gain = -(3/6)*log2(3/6) - (3/6)*log2(3/6) = 1
#    Tree3Total = 1.812


#    Tree4Left = 1 - (3/7)^2 - (4/7)^2 = 0.49
#    Gain = -(3/7)*log2(3/7) - (4/7)*log2(4/7) = 0.99

#    Tree4Right = 1 - (6/7)^2 - (1/7)^2 = 0.244
#    Gain = -(6/7)*log2(6/7) - (1/7)*log2(1.7) = 0.081
#    Tree4Total = 1.071

## We would want to choose Tree 2 because it has the highest information gain.

In [5]:
# importing libraries
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'webster-data445-bucket'
bucket = s3.Bucket(bucket_name)

# defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the csv file
heart = pd.read_csv(file_content_stream)
# dropping missing values
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
# defining the input and target variables
X = heart[['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

In [23]:
# creating list to store results
results = []

for i in range(0, 100):

    # splitting the data
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    # building the random forest
    RF1 = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)
    
    # extracting feature importance
    importances = RF1.feature_importances_
                                                         
    # appending our data
    results.append(importances)

In [24]:
# computing average feature importances
results = pd.DataFrame(results)
results.apply(np.mean, axis = 0)

## Our top 5 features are sysBP, BMI, totChol, age, glucose

0     0.022828
1     0.138353
2     0.041254
3     0.013192
4     0.055713
5     0.003789
6     0.021456
7     0.006907
8     0.140876
9     0.160933
10    0.148324
11    0.109870
12    0.136504
dtype: float64

In [32]:
# creating new train and test with our top 5 variables
X_train_rf = X_train[['sysBP', 'BMI', 'totChol', 'age', 'glucose']]
X_test_rf = X_test[['sysBP', 'BMI', 'totChol', 'age', 'glucose']]

In [37]:
# building the random forest with our top 5 variables and 3 max_depth
RF_md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train_rf, Y_train)

# predicting on test
pred1 = RF_md1.predict_proba(X_test_rf)[:, 1]
pred1 = np.where(pred1 < 0.1, 0, 1) # changing to labels

# computing recall
recall_score(Y_test, pred1)

0.7857142857142857

In [38]:
# building the random forest with our top 5 variables and 5 max depth
RF_md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_rf, Y_train)

# predicting on test
pred2 = RF_md2.predict_proba(X_test_rf)[:, 1]
pred2 = np.where(pred2 < 0.1, 0, 1) # changing to labels

# computing recall
recall_score(Y_test, pred2)

0.7767857142857143

In [40]:
# building the random forest with our top 5 variables and 7 max depth
RF_md3 = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train_rf, Y_train)

# predicting on test
pred3 = RF_md3.predict_proba(X_test_rf)[:, 1]
pred3 = np.where(pred3 < 0.1, 0, 1) # changing to labels

# computing recall
recall_score(Y_test, pred3)

0.75

In [46]:
# creating X2 and Y2
X2 = heart[['sysBP', 'BMI', 'totChol', 'age', 'glucose']]
Y2 = heart['TenYearCHD']

rf_md1_recall = []
rf_md2_recall = []
rf_md3_recall = []


for i in range(0, 100):
    
    # splitting into training and testing
    X_train, X_test, Y_train, Y_test = train_test_split(X2, Y2, test_size = 0.2, stratify = Y)
    
    ##### MODEL 1 
    
    # building the random forest with our top 5 variables and 3 max_depth
    rf_md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

    # predicting on test
    rf_md1_pred = rf_md1.predict_proba(X_test)[:, 1]
    rf_md1_pred = np.where(rf_md1_pred < 0.1, 0, 1) # changing to labels

    # computing recall and appending
    rf_md1_recall.append(recall_score(Y_test, rf_md1_pred))
    
    
    ##### MODEL 2
    
    # building the random forest with our top 5 variables and 5 max_depth
    rf_md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

    # predicting on test
    rf_md2_pred = rf_md2.predict_proba(X_test)[:, 1]
    rf_md2_pred = np.where(rf_md2_pred < 0.1, 0, 1) # changing to labels

    # computing recall and appending
    rf_md2_recall.append(recall_score(Y_test, rf_md2_pred))
    
    
    ##### MODEL 3
    
    # building the random forest with our top 5 variables and 7 max_depth
    rf_md3 = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train, Y_train)

    # predicting on test
    rf_md3_pred = rf_md3.predict_proba(X_test)[:, 1]
    rf_md3_pred = np.where(rf_md3_pred < 0.1, 0, 1) # changing to labels

    # computing recall and appending
    rf_md3_recall.append(recall_score(Y_test, rf_md3_pred))

In [61]:
# computing average recalls for the three models
md1_recall_results = pd.DataFrame(rf_md1_recall)
md2_recall_results = pd.DataFrame(rf_md2_recall)
md3_recall_results = pd.DataFrame(rf_md3_recall)

md1_final = md1_recall_results.apply(np.mean, axis = 0)
md2_final = md2_recall_results.apply(np.mean, axis = 0)
md3_final = md3_recall_results.apply(np.mean, axis = 0)

print('The recall score of the Random Forest with max_depth = 3 is:', md1_final.to_string(index = False))
print('The recall score of the Random Forest with max_depth = 5 is:', md2_final.to_string(index = False))
print('The recall score of the Random Forest with max_depth = 7 is:', md3_final.to_string(index = False))

## We would want to select the Random Forest with max_depth = 3 because it has the highest recall score

The recall score of the Random Forest with max_depth = 3 is:  0.840179
The recall score of the Random Forest with max_depth = 5 is:  0.82375
The recall score of the Random Forest with max_depth = 7 is:  0.805982
